In [1]:
import time
import numpy as np
import torch

from get_args import get_args
from agent import Agent
from utils import load_filepath, load_checkpoint, plot_scores, print_debug_info, print_status, save_checkpoint
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
from unityagents import UnityEnvironment

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

args = get_args()
args.framework = "D2DQN"
args.pixels = True

In [2]:
for arg in vars(args):
    print("{}: {}".format(arg, getattr(args, arg)))

framework: D2DQN
prioritized_replay: False
alpha: 0.4
batchsize: 64
buffersize: 100000
c: 8
continue: False
debug: False
dropout: 0.05
epsilon: 1.0
epsilon_decay: 0.999
epsilon_min: 0.075
gamma: 0.99
latest: False
learn_rate: 0.00025
momentum: 0.95
nographics: False
num_episodes: 1500
optimizer: Adam
pixels: True
print_count: 15
tau: 0.001
train: False
update_every: 4
verbose: False


In [3]:
start_time = time.time()
sep = "#"*50
unity_env = UnityEnvironment(file_name="VisualBanana_Windows_x86_64/Banana.exe", no_graphics=True)
# get the default brain (In this environment there is only one agent/brain)
brain_name = unity_env.brain_names[0]
brain = unity_env.brains[brain_name]
env = unity_env.reset(train_mode=True)[brain_name]
nA = brain.vector_action_space_size
nS = len(env.vector_observations[0])
agent = Agent(nS, nA, device, args)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 1
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 0
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Using Pixel-based training.


In [ ]:
state = env.visual_observations[0]
print(state.shape)
state = torch.from_numpy(state)
print(state.shape)

In [ ]:
state = env.visual_observations[0]
state = torch.from_numpy(state).squeeze(0).transpose(2,0)
#state = state
state.shape

In [ ]:
state = env.visual_observations[0]
state = state.squeeze(0).transpose(2,0,1)
state.shape

In [4]:
#state = torch.from_numpy(env.visual_observations[0]).float().transpose(3,1).to(device)
state = env.visual_observations[0]

for i in range(60):    
    print(state.shape)
    #action = agent.act(state.squeeze(0).transpose(2,0,1))
    action = agent.act(state)
    env = unity_env.step(action)[brain_name]

    #collect info about new state
    reward = env.rewards[0]
    next_state = env.visual_observations[0]
    done = env.local_done[0]
    #action = torch.from_numpy(action)
    agent.teststep(state, action, reward, next_state, done)
    agent.buffer.add(state, action, reward, next_state, done)

    state = next_state
    

(1, 84, 84, 3)


TypeError: gather(): argument 'index' (position 2) must be Tensor, not numpy.int32

In [ ]:
batch = agent.buffer.sample()
#batch = state, action, reward, next_state, done

In [ ]:
for state, action, reward, next_state, done in zip(*batch):
    print("STATE:\n{}".format(state))
    print("ACTION:\n{}".format(action))
    print("REWARD:\n{}".format(reward))
    print("NEXT STATE:\n{}".format(next_state))
    print("DONE:\n{}".format(done))    

In [ ]:
print("STATE:\n{}".format(state))
print("ACTION:\n{}".format(action))
print("REWARD:\n{}".format(reward))
print("NEXT STATE:\n{}".format(next_state))
print("DONE:\n{}".format(done))

In [ ]:
#Binary tree
class Node:
    def __init__(self,data)

In [ ]:
unity_env.close()
